In [0]:
account = "goodreadsreviews60106541" 
key = "<AccoutnKey>"

spark.conf.set(f"fs.azure.account.key.{account}.dfs.core.windows.net", key)


In [0]:
account = "goodreadsreviews60106541"
container = "lakehouse"

display(dbutils.fs.ls(f"abfss://{container}@{account}.dfs.core.windows.net/"))


path,name,size,modificationTime
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/,gold/,0,1762463595000
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed/,processed/,0,1762430158000
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/raw/,raw/,0,1762084980000


In [0]:
path = f"abfss://{container}@{account}.dfs.core.windows.net/gold/"
df = spark.read.parquet(path)
df.show(5)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8762651512584079>, line 2
      1 path = f"abfss://{container}@{account}.dfs.core.windows.net/gold/"
----> 2 df = spark.read.parquet(path)
      3 df.show(5)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:641, in DataFrameReader.parquet(self, *paths, **options)
    630 int96RebaseMode = options.get("int96RebaseMode", None)
    631 self._set_opts(
    632     mergeSchema=mergeSchema,
    633     pathGlobFilter=pathGlobFilter,
   (...)
    638     i

In [0]:
display(dbutils.fs.ls(f"abfss://{container}@{account}.dfs.core.windows.net/"))


path,name,size,modificationTime
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/,gold/,0,1762463595000
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed/,processed/,0,1762430158000
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/raw/,raw/,0,1762084980000


In [0]:
display(dbutils.fs.ls("abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/"))


path,name,size,modificationTime
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/curated_reviews/,curated_reviews/,0,1762463595000
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/curated_reviews_delta/,curated_reviews_delta/,0,1762502049000


In [0]:
df = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/curated_reviews_delta/"
)
df.show(5)


+--------------------+--------+--------------------+---------+-----------------+--------------------+------+--------------------+--------+-------+----------+
|           review_id| book_id|               title|author_id|             name|             user_id|rating|         review_text|language|n_votes|date_added|
+--------------------+--------+--------------------+---------+-----------------+--------------------+------+--------------------+--------+-------+----------+
|8efa123caf94e9fef...|21853678|The Secrets of Mi...|  3271990|    Thomas LaRock|cd23d9de8fb9983fe...|     4|Short and sweet, ...|     eng|   7301|2015-02-10|
|16aadd44271c9a4ef...|  289695|Seeing Redd (The ...|  1540408|    Debra Ullrick|8bf745a1e2b3ec721...|     3|I found this book...|   en-US|  14658|      NULL|
|333d06c9cffd9599d...| 7619398|American Vampire,...|  5950800|Boniface Verheyen|641a6c38e46235ed6...|     5|The image of the ...|     eng|  17927|2010-10-05|
|52b5e1f96d8d21729...| 1131779|Encyclopedia of G...|

In [0]:
df.printSchema()     # see column names and types
df.count()           # total number of rows
df.describe().show() # quick stats


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: date (nullable = true)

+-------+--------------------+-------------------+--------------+-----------------+----------+--------------------+------------------+--------------------+--------+------------------+
|summary|           review_id|            book_id|         title|        author_id|      name|             user_id|            rating|         review_text|language|           n_votes|
+-------+--------------------+-------------------+--------------+-----------------+----------+--------------------+------------------+--------------------+--------+------------------+
|  coun

In [0]:
from pyspark.sql.functions import col, trim, lower, length
from pyspark.sql import functions as F

# Remove rows with missing essential fields
df_clean = df.dropna(subset=["rating", "review_text", "book_id", "author_id"])

# Remove duplicates (keep first occurrence)
df_clean = df_clean.dropDuplicates(["review_id"]) \
                   .dropDuplicates(["user_id", "book_id"])

# Normalize text fields
df_clean = (
    df_clean
    .withColumn("review_text", trim(lower(col("review_text"))))
    .withColumn("review_text", F.regexp_replace(col("review_text"), "[^a-zA-Z0-9\\s,.!?]", "")) # remove malformed chars
)

# Drop reviews with very short text (less than 10 chars for example)
df_clean = df_clean.filter(length(col("review_text")) > 10)

# Correct data types
df_clean = (
    df_clean
    .withColumn("rating", col("rating").cast("float"))
    .withColumn("book_id", col("book_id").cast("string"))
    .withColumn("user_id", col("user_id").cast("string"))
    .withColumn("author_id", col("author_id").cast("string"))
)

# Drop unused / irrelevant columns (example: "timestamp", "url", etc.)
cols_to_drop = ["timestamp", "url", "extra_info"]
for c in cols_to_drop:
    if c in df_clean.columns:
        df_clean = df_clean.drop(c)

# verify numeric columns are valid (e.g., rating between 0 and 5)
df_clean = df_clean.filter((col("rating") >= 0) & (col("rating") <= 5))

# Show sample
df_clean.show(5, truncate=False)



+--------------------------------+--------+---------------------------------------------------+---------+-----------------+--------------------------------+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import size, split, avg, count

# Compute review length in words
df_features = df_clean.withColumn(
    "review_length_words", size(split(col("review_text"), "\\s+"))
)

df_features.select("review_text", "review_length_words").show(5, truncate=80)


+--------------------------------------------------------------------------------+-------------------+
|                                                                     review_text|review_length_words|
+--------------------------------------------------------------------------------+-------------------+
|so, i had to jump on the crossover madness, and even though ive often been a ...|                 91|
|ive never read a christmas carol gasp but ive seen almost every version there...|                158|
|i loved this book. it is slow to start. if you give it a chance, you will sta...|                 36|
|                                                     omg omg omg omg that is all|                  7|
|despite a slow start, this book kept me turning pages right until the surpris...|                 34|
+--------------------------------------------------------------------------------+-------------------+
only showing top 5 rows


In [0]:
agg_df = (
    df_features.groupBy("book_id")
    .agg(
        avg("rating").alias("avg_rating"),
        count("review_id").alias("num_reviews")
    )
)

agg_df.show(5)


+--------+------------------+-----------+
| book_id|        avg_rating|num_reviews|
+--------+------------------+-----------+
|   63697|3.7316239316239317|        585|
| 6679629|               4.0|          1|
|33386293|               3.0|          1|
|  486298| 3.912280701754386|         57|
|32620360| 4.293333333333333|        150|
+--------+------------------+-----------+
only showing top 5 rows


In [0]:
df_enriched = df_features.join(agg_df, on="book_id", how="left")
df_enriched.show(5, truncate=80)


+--------+--------------------------------+------------------------------------+---------+-------------------+--------------------------------+------+--------------------------------------------------------------------------------+--------+-------+----------+-------------------+------------------+-----------+
| book_id|                       review_id|                               title|author_id|               name|                         user_id|rating|                                                                     review_text|language|n_votes|date_added|review_length_words|        avg_rating|num_reviews|
+--------+--------------------------------+------------------------------------+---------+-------------------+--------------------------------+------+--------------------------------------------------------------------------------+--------+-------+----------+-------------------+------------------+-----------+
|23590496|dd07a9093f54b5521818bd9e38d6c3dc|                      Fo

In [0]:
output_path = "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/features_v1/"

df_enriched.write.format("delta").mode("overwrite").save(output_path)
print("features_v1 saved successfully to Gold layer")


features_v1 saved successfully to Gold layer


In [0]:
df_check = spark.read.format("delta").load(output_path)

print("Rows:", df_check.count())
df_check.printSchema()
df_check.show(5, truncate=80)

Rows: 14913947
root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- num_reviews: long (nullable = true)

+--------+--------------------------------+-------------------------------+---------+---------------------+--------------------------------+------+--------------------------------------------------------------------------------+--------+-------+----------+-------------------+------------------+-----------+
| book_id|                       review_id|                          title|author_id|                 n